## Test running BQ ML from jupyter notebook

# NOTE: This code needs to be updated now that it has been pulled out of the model_train_save_test.ipynb

In [4]:
import os
import pandas as pd
# Allow you to easily have Python variables in SQL query.
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

### Create Project and Bucket name vars

In [5]:
bucket_path = os.path.expanduser('~/bucket.txt')
with open(bucket_path) as f:
    BUCKET_NAME = f.read().strip()

    project_path = os.path.expanduser('~/project.txt')
with open(project_path) as f:
    PROJECT_NAME = f.read().strip()

In [6]:
@register_cell_magic('with_globals')
def with_globals(line, cell):
    contents = cell.format(**globals())
    if 'print' in line:
        print(contents)
    get_ipython().run_cell(contents)

In [8]:
%%with_globals
%%bigquery --project $PROJECT_NAME

# ToDo: Update this so it's not dependent on another model run that does 
# this mapping. In other words, we should do any mapping directly in this code. 

#Create Model - Dataset being used is the public dataset from above with the encoders 
CREATE or REPLACE MODEL aiplatform_test.liquor_sales_model
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['sale_dollars']) AS
    SELECT sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat

Executing query with job ID: ca42d882-bed7-4f58-bbb8-22e14ca3f56c
Query executing: 0.22s


ERROR:
 404 Not found: Dataset ipt-dev-888b4e:aiplatform_test was not found in location US

(job ID: ca42d882-bed7-4f58-bbb8-22e14ca3f56c)

                                                              -----Query Job SQL Follows-----                                                               

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:# ToDo: Update this so it's not dependent on another model run that does this mapping. In other words, we should do any mapping directly in this code. 
   2:#Create Model - Dataset being used is the public dataset from above with the encoders 
   3:CREATE or REPLACE MODEL aiplatform_test.liquor_sales_model
   4:OPTIONS
   5:  (model_type='linear_reg',
   6:    input_label_cols=['sale_dollars']) AS
   7:    SELECT sale_dollars,
   8:     city_enc,
   9:     county_number_enc,
  10:     category_enc,
  11:     store_number_enc,
  12:   

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Evaluate Model 
SELECT
    *
FROM
  ML.EVALUATE(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat))

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Prediction Outcomes
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
))

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Create a table with the results 
CREATE TABLE aiplatform_test.bqml_liquor_sales AS (
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
)))